# 导包

In [3]:
import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

print(device)


cpu


# 数据

In [9]:
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False),
])

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=transform)
# After applying the `RandomLinkSplit` transform, the data is transformed from
# a data object to a list of tuples (train_data, val_data, test_data), with
# each element representing the corresponding split.
train_data, val_data, test_data = dataset[0]

Processing...
Done!


# 模型

In [13]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

# 训练

In [14]:
model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss

# 测试

In [16]:
@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)
print(z)
print(final_edge_index)

Epoch: 001, Loss: 0.4457, Val: 0.9020, Test: 0.8953
Epoch: 002, Loss: 0.4450, Val: 0.9023, Test: 0.8950
Epoch: 003, Loss: 0.4421, Val: 0.9019, Test: 0.8948
Epoch: 004, Loss: 0.4478, Val: 0.9018, Test: 0.8965
Epoch: 005, Loss: 0.4454, Val: 0.9018, Test: 0.8978
Epoch: 006, Loss: 0.4414, Val: 0.9034, Test: 0.9000
Epoch: 007, Loss: 0.4359, Val: 0.9041, Test: 0.9011
Epoch: 008, Loss: 0.4411, Val: 0.9037, Test: 0.9012
Epoch: 009, Loss: 0.4409, Val: 0.9025, Test: 0.9013
Epoch: 010, Loss: 0.4410, Val: 0.9034, Test: 0.9023
Epoch: 011, Loss: 0.4341, Val: 0.9045, Test: 0.9039
Epoch: 012, Loss: 0.4315, Val: 0.9042, Test: 0.9053
Epoch: 013, Loss: 0.4442, Val: 0.9025, Test: 0.9055
Epoch: 014, Loss: 0.4338, Val: 0.9015, Test: 0.9055
Epoch: 015, Loss: 0.4339, Val: 0.9024, Test: 0.9062
Epoch: 016, Loss: 0.4413, Val: 0.9024, Test: 0.9065
Epoch: 017, Loss: 0.4295, Val: 0.9035, Test: 0.9074
Epoch: 018, Loss: 0.4359, Val: 0.9029, Test: 0.9074
Epoch: 019, Loss: 0.4403, Val: 0.9023, Test: 0.9074
Epoch: 020, 